# FlowerPower IO Library Demo

This notebook demonstrates the key features of the FlowerPower IO library, including:

1. **Reading CSV files** using `CSVFileReader`
2. **Converting data** to different formats (Pandas DataFrame, Polars DataFrame, PyArrow Table)
3. **Writing to Parquet** using `ParquetFileWriter`
4. **Reading from SQLite database** using `SQLiteReader`
5. **Writing to SQLite database** using `SQLiteWriter`

Let's start by importing the necessary classes and creating some sample data.

In [ ]:
# Import required libraries
import pandas as pd
import polars as pl
import pyarrow as pa
import tempfile
import os
from pathlib import Path

# Import FlowerPower IO classes
from flowerpower_io.loader.csv import CSVFileReader
from flowerpower_io.saver.parquet import ParquetFileWriter
from flowerpower_io.loader.sqlite import SQLiteReader
from flowerpower_io.saver.sqlite import SQLiteWriter

print("Successfully imported all required classes!")

## 1. Creating Sample Data

First, let's create a sample CSV file that we'll use throughout this demonstration.

In [ ]:
# Create sample data
sample_data = {
    'id': range(1, 101),
    'name': [f'Person_{i}' for i in range(1, 101)],
    'age': [20 + (i % 50) for i in range(1, 101)],
    'city': ['New York', 'London', 'Tokyo', 'Paris', 'Berlin'] * 20,
    'salary': [50000 + (i * 1000) for i in range(1, 101)]
}

# Create a temporary directory for our demo files
temp_dir = tempfile.mkdtemp()
csv_path = os.path.join(temp_dir, 'sample_data.csv')
parquet_path = os.path.join(temp_dir, 'sample_data.parquet')
db_path = os.path.join(temp_dir, 'sample_data.db')

# Create CSV file using pandas
df_pandas = pd.DataFrame(sample_data)
df_pandas.to_csv(csv_path, index=False)

print(f"Created sample CSV file at: {csv_path}")
print(f"Sample data shape: {df_pandas.shape}")
print("\nFirst 5 rows:")
print(df_pandas.head())

## 2. Reading CSV Files with CSVFileReader

Now let's demonstrate how to read the CSV file using `CSVFileReader` and convert it to different formats.

In [ ]:
# Initialize CSVFileReader
csv_reader = CSVFileReader(path=csv_path)

print("CSVFileReader initialized successfully!")
print(f"File path: {csv_reader.path}")
print(f"Format: {csv_reader.format}")

## 3. Converting to Different Data Formats

The `CSVFileReader` can convert data to multiple formats including Pandas DataFrame, Polars DataFrame, and PyArrow Table.

In [ ]:
# Convert to Pandas DataFrame
print("=== Converting to Pandas DataFrame ===")
df_pandas_converted = csv_reader.to_pandas()
print(f"Pandas DataFrame shape: {df_pandas_converted.shape}")
print(f"Data types:\n{df_pandas_converted.dtypes}")
print("\nFirst 3 rows:")
print(df_pandas_converted.head(3))

In [ ]:
# Convert to Polars DataFrame
print("\n=== Converting to Polars DataFrame ===")
df_polars = csv_reader.to_polars()
print(f"Polars DataFrame shape: {df_polars.shape}")
print(f"Schema: {df_polars.schema}")
print("\nFirst 3 rows:")
print(df_polars.head(3))

In [ ]:
# Convert to PyArrow Table
print("\n=== Converting to PyArrow Table ===")
arrow_table = csv_reader.to_pyarrow_table()
print(f"PyArrow Table shape: {arrow_table.shape}")
print(f"Schema: {arrow_table.schema}")
print("\nFirst 3 rows:")
print(arrow_table.slice(0, 3).to_pandas())

## 4. Writing to Parquet with ParquetFileWriter

Now let's demonstrate how to write data to a Parquet file using `ParquetFileWriter`.

In [ ]:
# Initialize ParquetFileWriter
parquet_writer = ParquetFileWriter(path=parquet_path)

print("ParquetFileWriter initialized successfully!")
print(f"Output path: {parquet_writer.path}")
print(f"Format: {parquet_writer.format}")

In [ ]:
# Write data to Parquet file using Pandas DataFrame
print("=== Writing Pandas DataFrame to Parquet ===")
metadata = parquet_writer.write(df_pandas_converted)
print(f"Write operation completed!")
print(f"Metadata: {metadata}")

# Check if file was created
print(f"\nParquet file exists: {os.path.exists(parquet_path)}")
print(f"File size: {os.path.getsize(parquet_path)} bytes")

In [ ]:
# Let's also try writing with Polars DataFrame
print("\n=== Writing Polars DataFrame to Parquet ===")
parquet_path_polars = os.path.join(temp_dir, 'sample_data_polars.parquet')
parquet_writer_polars = ParquetFileWriter(path=parquet_path_polars)

metadata_polars = parquet_writer_polars.write(df_polars)
print(f"Write operation completed!")
print(f"Metadata: {metadata_polars}")
print(f"File size: {os.path.getsize(parquet_path_polars)} bytes")

## 5. Reading from SQLite Database with SQLiteReader

Now let's demonstrate how to write data to a SQLite database and then read it back using `SQLiteWriter` and `SQLiteReader`.

In [ ]:
# First, let's write data to SQLite database using SQLiteWriter
print("=== Writing to SQLite Database ===")
sqlite_writer = SQLiteWriter(
    table_name="employees",
    path=db_path
)

print("SQLiteWriter initialized successfully!")
print(f"Database path: {sqlite_writer.path}")
print(f"Table name: {sqlite_writer.table_name}")
print(f"Type: {sqlite_writer.type_}")

In [ ]:
# Write the data to SQLite
write_metadata = sqlite_writer.write(df_pandas_converted)
print(f"Data written to SQLite successfully!")
print(f"Write metadata: {write_metadata}")

# Verify database file was created
print(f"\nDatabase file exists: {os.path.exists(db_path)}")
print(f"Database file size: {os.path.getsize(db_path)} bytes")

In [ ]:
# Now let's read the data back using SQLiteReader
print("\n=== Reading from SQLite Database ===")
sqlite_reader = SQLiteReader(
    table_name="employees",
    path=db_path
)

print("SQLiteReader initialized successfully!")
print(f"Database path: {sqlite_reader.path}")
print(f"Table name: {sqlite_reader.table_name}")
print(f"Type: {sqlite_reader.type_}")

In [ ]:
# Read data as Pandas DataFrame
print("=== Reading as Pandas DataFrame ===")
df_from_sqlite_pandas = sqlite_reader.to_pandas()
print(f"Data shape: {df_from_sqlite_pandas.shape}")
print("\nFirst 5 rows:")
print(df_from_sqlite_pandas.head())

# Verify data integrity
print(f"\nData integrity check - Original vs SQLite:")
print(f"Original shape: {df_pandas_converted.shape}")
print(f"SQLite shape: {df_from_sqlite_pandas.shape}")
print(f"Data matches: {df_pandas_converted.equals(df_from_sqlite_pandas)}")

In [ ]:
# Read data as Polars DataFrame
print("\n=== Reading as Polars DataFrame ===")
df_from_sqlite_polars = sqlite_reader.to_polars()
print(f"Data shape: {df_from_sqlite_polars.shape}")
print("\nFirst 5 rows:")
print(df_from_sqlite_polars.head())

# Read data as PyArrow Table
print("\n=== Reading as PyArrow Table ===")
arrow_from_sqlite = sqlite_reader.to_pyarrow_table()
print(f"Data shape: {arrow_from_sqlite.shape}")
print("\nFirst 5 rows:")
print(arrow_from_sqlite.slice(0, 5).to_pandas())

## 6. Advanced Querying with SQLiteReader

Let's demonstrate how to use custom SQL queries with the SQLiteReader.

In [ ]:
# Query for employees older than 50
print("=== Custom SQL Query: Employees older than 50 ===")
query = "SELECT * FROM employees WHERE age > 50"
df_older_employees = sqlite_reader.to_pandas(query=query)
print(f"Number of employees older than 50: {len(df_older_employees)}")
print("\nEmployees older than 50:")
print(df_older_employees)

In [ ]:
# Query for average salary by city
print("\n=== Custom SQL Query: Average salary by city ===")
query = "SELECT city, AVG(salary) as avg_salary, COUNT(*) as count FROM employees GROUP BY city ORDER BY avg_salary DESC"
df_salary_by_city = sqlite_reader.to_pandas(query=query)
print("Average salary by city:")
print(df_salary_by_city)

## 7. Metadata and Performance Information

Let's explore the metadata functionality and get some performance insights.

In [ ]:
# Get metadata from CSV reader
print("=== CSV Reader Metadata ===")
df_pandas_with_metadata, csv_metadata = csv_reader.to_pandas(metadata=True)
print(f"CSV Metadata: {csv_metadata}")

# Get metadata from SQLite reader
print("\n=== SQLite Reader Metadata ===")
df_sqlite_with_metadata, sqlite_metadata = sqlite_reader.to_pandas(metadata=True)
print(f"SQLite Metadata: {sqlite_metadata}")

In [ ]:
# Compare file sizes
print("\n=== File Size Comparison ===")
csv_size = os.path.getsize(csv_path)
parquet_size = os.path.getsize(parquet_path)
db_size = os.path.getsize(db_path)

print(f"CSV file size: {csv_size:,} bytes")
print(f"Parquet file size: {parquet_size:,} bytes")
print(f"SQLite database size: {db_size:,} bytes")
print(f"\nCompression ratios:")
print(f"Parquet vs CSV: {csv_size/parquet_size:.2f}x smaller")
print(f"SQLite vs CSV: {csv_size/db_size:.2f}x smaller")

## 8. Cleanup

Let's clean up the temporary files we created during this demonstration.

In [ ]:
# Clean up temporary files
import shutil

print("=== Cleaning up temporary files ===")
files_to_remove = [csv_path, parquet_path, parquet_path_polars, db_path]

for file_path in files_to_remove:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Removed: {file_path}")

# Remove temporary directory
shutil.rmtree(temp_dir)
print(f"\nRemoved temporary directory: {temp_dir}")

print("\nCleanup completed successfully!")

## Summary

This notebook demonstrated the key features of the FlowerPower IO library:

1. **CSV Reading**: Used `CSVFileReader` to read CSV files and convert them to multiple formats
2. **Data Conversion**: Showed how to convert between Pandas, Polars, and PyArrow formats
3. **Parquet Writing**: Used `ParquetFileWriter` to save data in the efficient Parquet format
4. **Database Operations**: Demonstrated both reading from and writing to SQLite databases using `SQLiteReader` and `SQLiteWriter`
5. **Advanced Querying**: Showed how to use custom SQL queries for data filtering and aggregation
6. **Metadata**: Explored metadata functionality to get insights about the data

The FlowerPower IO library provides a unified interface for various data operations, making it easy to work with different file formats and database systems while maintaining excellent performance and flexibility.